In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/")
import data_preparation as dp
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings("ignore", category=FutureWarning)
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Import des données

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_26526/176984250.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/train.csv")
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_26526/176984250.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("../data/test.csv")


In [3]:
preprocess = dp.DataPreparation(train, test)

In [4]:
train_clean, test_clean = preprocess.prepare_data()

Valeurs manquantes du train supprimées ✅
Valeurs manquantes du test supprimées ✅
Variables renommées ✅
Valeurs manquantes numériques imputées ✅
Valeurs manquantes catégorielles imputées ✅


In [5]:
X_train = train_clean.drop(columns = ["Ewltp (g/km)"])
y_train = train_clean["Ewltp (g/km)"]

KeyError: "['Ewltp (g/km)'] not found in axis"

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
numericals = train_clean.select_dtypes(exclude='object').columns.to_list()
categoricals = train_clean.select_dtypes(include='object').columns.to_list()

numericals.remove('ID')
numericals.remove('date')

# Choix des variables à inclure

In [ ]:
y_corr = train[numericals].corr()
sns.heatmap(y_corr, annot=True)

In [ ]:
correlation_vector = train[numericals].corr()["Ewltp (g/km)"][:]
correlation_vector = np.abs(correlation_vector)
correlation_vector = correlation_vector.sort_values(ascending=False)[1:]

In [ ]:
var_explicatives = correlation_vector.index[:4].to_list()

# Modélisation

In [ ]:
reg = LinearRegression().fit(X_train[var_explicatives], y_train)

In [ ]:
pred = reg.predict(X_val[var_explicatives])

In [ ]:
mae = mean_absolute_error(y_val, pred)

In [ ]:
mae

# Test

In [ ]:
test_pred = reg.predict(test_clean[var_explicatives])

In [ ]:
sub = test[["ID", 'ec (cm3)']]

In [ ]:
sub["Ewltp (g/km)"] = test_pred

In [ ]:
sub.drop(columns=['ec (cm3)'], inplace = True)

In [ ]:
sub.to_csv("../data/sample_submission2.csv", index=False)

# Correlation catégorielle

In [6]:
X_train = train_clean.drop(columns=["Ewltp_(g/km)"])
y_train = train_clean["Ewltp_(g/km)"]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [7]:
train_clean = train_clean.rename(columns={"Ewltp_(g/km)": "Ewltp_g_km"})

In [9]:
formula = 'Ewltp_g_km ~ ' + ' + '.join(['C(' + var + ')' for var in preprocess.col_categoricals])

model = ols(formula, data=train_clean.sample(10000)).fit()
aov_table = sm.stats.anova_lm(model, typ=2) 

print('--------------------------- Table Anova ---------------------------')
print(aov_table)

print('--------------------------- Eta Carré ---------------------------')
eta = [aov_table['sum_sq'][i] / sum(aov_table['sum_sq']) for i in range(len(preprocess.col_categoricals))]
for i in range(len(preprocess.col_categoricals)):
    print(f'{preprocess.col_categoricals[i]} : {round(eta[i], 2)*100}')

/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 28, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1776, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/axel/Documents/École/Universit

--------------------------- Table Anova ---------------------------
                   sum_sq      df         F    PR(>F)
C(Country)     503.571603    28.0  0.135251  0.713065
C(VFN)       32075.311878  1776.0  0.135820  0.712488
C(Mh)         1079.082008    60.0  0.135251  0.713065
C(Man)        1061.097308    59.0  0.135251  0.713065
C(Tan)       28708.678112  1571.0  0.137428  0.710868
C(T)          6646.807800   370.0  0.135098  0.713220
C(Va)        23969.805190  1331.0  0.135433  0.712880
C(Ve)        55003.502661  3073.0  0.134606  0.713719
C(Mk)         1600.638312    89.0  0.135251  0.713065
C(Cn)        20662.707767  1165.0  0.133382  0.714966
C(Ct)           17.984700     1.0  0.135251  0.713065
C(Cr)           17.984700     1.0  0.135251  0.713065
C(Ft)          161.862301     9.0  0.135251  0.713065
C(Fm)           89.923501     5.0  0.135251  0.713065
Residual    644120.312369  4844.0       NaN       NaN
--------------------------- Eta Carré ---------------------------
Co

/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/statsmodels/base/model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '


# Feature Engineering

In [ ]:
vehicles = pd.read_csv("../data/all-vehicles-model.csv", sep = ';')

In [ ]:
vehicles

In [ ]:
modeles = list(vehicles["Model"].unique())

In [ ]:
import Levenshtein
from tqdm import tqdm
modeles2 = list(train_clean["Cn"].unique())
commun = {}
for element in tqdm(modeles2) : 
    commun[element]=[]
    similarity_score = {element2 : Levenshtein.ratio(element, element2) for element2 in modeles}
    for keys, value in similarity_score.items():
        if value > 0.6 :
            commun[element].append(vehicles[vehicles["Model"]==keys]["Vehicle Size Class"].to_list()[0])


In [ ]:
commun

In [ ]:
from statistics import mode

commun_red = {keys : mode(value)for keys,value in commun.items() if len(value) > 0}

In [ ]:
len(commun_red)

In [ ]:
train_clean["Cn"].nunique()